In [31]:
!pip install --upgrade torchvision

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 96.1 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.8/899.8 MB 56.0 MB/s  0:00:07m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 56.1 MB/s  0:00:06m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 130.0 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 MB 131.8 MB/s  0:00:00m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 954.8/954.8 kB 8.7 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 706.8/706.8 MB 62.4 MB/s  0:00:06m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.1/193.1 MB 125.5 MB/s  0:00:010:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 132.0 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.6/63.6 MB 169.9 MB/s  0:00:00 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.5/267.5 MB 76.3 MB/s  0:00:03m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.2/288.2 MB 128.

In [32]:
# =========================
# Imports
# =========================
import os
import numpy as np
import torch
from collections import defaultdict

from torchvision.io import read_image, write_jpeg
from torchvision.utils import draw_bounding_boxes

from src.stores import ModelProvidersFactory, DatasetProvidersFactory
from src.utils.checkpoints_utils import load_checkpoint
from src.Preprocessing.volleyball_annot_loader import load_tracking_annot


# =========================
# Constants / Dictionaries
# =========================
CATEGORIES_DICT = {
    'l-pass': 0,
    'r-pass': 1,
    'l-spike': 2,
    'r_spike': 3,
    'l_set': 4,
    'r_set': 5,
    'l_winpoint': 6,
    'r_winpoint': 7
}

ACTIONS_DICT = {
    "waiting": 0,
    "setting": 1,
    "digging": 2,
    "falling": 3,
    "spiking": 4,
    "blocking": 5,
    "jumping": 6,
    "moving": 7,
    "standing": 8
}

ID_TO_CATEGORY = {v: k for k, v in CATEGORIES_DICT.items()}


# =========================
# Paths
# =========================
VIDEO_ROOT = "data/volleyball/volleyball_/videos/4"
TRACK_ROOT = "data/volleyball/volleyball_tracking_annotation/volleyball_tracking_annotation/4"
GROUP_GT_PATH = "data/volleyball/volleyball_/videos/4/annotations.txt"
SAVE_ROOT = "./demo/4"

os.makedirs(SAVE_ROOT, exist_ok=True)

In [33]:
# =========================
# Load group GT annotations
# =========================
from src.Preprocessing.boxinfo import BoxInfo
def load_group_annotations(path):
    """
    Returns:
        { clip_name (str) : group_label_id (int) }
    """
    clip_gt = {}
    with open(path, "r") as f:
        for line in f:
            parts = line.strip().split()
            clip_name = parts[0].replace(".jpg", "")
            group_label = parts[1]
            clip_gt[clip_name] = CATEGORIES_DICT[group_label]
    return clip_gt


# =========================
# Load tracking annotations
# (your exact logic preserved)
# =========================
def load_tracking_annot(path):
    with open(path, 'r') as file:
        player_boxes = {idx: [] for idx in range(12)}
        frame_boxes_dct = {}

        for line in file:
            box_info = BoxInfo(line)
            if box_info.player_ID > 11:
                continue
            player_boxes[box_info.player_ID].append(box_info)

        # build frame → boxes view
        for player_ID, boxes_info in player_boxes.items():
            # keep middle frames only (as in your code)
            boxes_info = boxes_info[5:]
            boxes_info = boxes_info[:-6]

            for box_info in boxes_info:
                if box_info.frame_ID not in frame_boxes_dct:
                    frame_boxes_dct[box_info.frame_ID] = []
                frame_boxes_dct[box_info.frame_ID].append(box_info)

        return frame_boxes_dct

In [34]:
# =========================
# Device
# =========================
device = "cuda" if torch.cuda.is_available() else "cpu"


# =========================
# Load GT
# =========================
group_gt = load_group_annotations(GROUP_GT_PATH)


# =========================
# Create dataloader (for consistency with training)
# =========================
dataset_factory = DatasetProvidersFactory()

_, testloader = dataset_factory.get_data_loaders(
    4,
    "data/volleyball/volleyball_/videos",
    "data/volleyball/volleyball_tracking_annotation/volleyball_tracking_annotation",
    [0],
    [4],
    True,
    "logs/baselines_logs/baseline8_logs",
    CATEGORIES_DICT,
    "output/player_level_features",
    False,
    True,
    True,
    True
)


# =========================
# Create model (MATCH TRAINING CONFIG)
# =========================
models_factory = ModelProvidersFactory()

model = models_factory.create(
    model_name="b8",
    num_classes=8,
    input_size=2048,     # 🔥 MUST match checkpoint
    hidden_size1=1024,
    hidden_size2=512,
    num_layers=1,
    log_dir="logs/baselines_logs/baseline8_logs",
    verbose=True
)

checkpoint = torch.load("models/b8/best_model.pth", map_location="cpu")
model = load_checkpoint(checkpoint=checkpoint, model=model)

model.to(device)
model.eval()


[2025-12-27 22:57:01] - [FeaturesDataset.py:54] - [INFO]: Initializing Features Dataset...
[2025-12-27 22:57:01] - [FeaturesDataset.py:74] - [INFO]: Collecting Images' features and labels...
[2025-12-27 22:57:01] - [FeaturesDataset.py:56] - [INFO]: Features Dataset Initialized with 153 Samples!
[2025-12-27 22:57:01] - [FeaturesDataset.py:54] - [INFO]: Initializing Features Dataset...
[2025-12-27 22:57:01] - [FeaturesDataset.py:74] - [INFO]: Collecting Images' features and labels...
[2025-12-27 22:57:01] - [FeaturesDataset.py:56] - [INFO]: Features Dataset Initialized with 100 Samples!
[2025-12-27 22:57:01] - [baseline8.py:61] - [INFO]: Initializing Two Stage Activity Temporal Classifier...
[2025-12-27 22:57:01] - [baseline8.py:85] - [INFO]: Model Initialized Successfully!


Two_Stage_Pooled_Teams_Activity_Temporal_Classifier(
  (player_lstm): LSTM(2048, 1024, batch_first=True)
  (adaptive_max_pool): AdaptiveMaxPool1d(output_size=1)
  (frame_lstm): LSTM(2048, 512, batch_first=True)
  (fc): Sequential(
    (0): Linear(in_features=512, out_features=512, bias=True)
    (1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.5, inplace=False)
    (4): Linear(in_features=512, out_features=8, bias=True)
  )
)

In [35]:
# =========================
# Visualization + inference
# =========================
from PIL import ImageFont

for clip in sorted(os.listdir(VIDEO_ROOT)):
    clip_path = os.path.join(VIDEO_ROOT, clip)
    track_file = os.path.join(TRACK_ROOT, clip, f"{clip}.txt")

    if not os.path.exists(track_file):
        continue

    frames_ann = load_tracking_annot(track_file)
    print(f"[DEBUG] clip={clip}, frames={len(frames_ann)}")

    save_clip = os.path.join(SAVE_ROOT, clip)
    os.makedirs(save_clip, exist_ok=True)

    # ---- group prediction (clip-level) ----
    with torch.no_grad():
        for data, target in testloader:

            if isinstance(data, list):
                data, target = np.array(data), np.array(target)

            data = torch.tensor(data, dtype=torch.float32).to(device)
            target = torch.tensor(target, dtype=torch.long).to(device)

            # Forward
            logits = model(data)

            # Predictions
            preds = torch.argmax(logits, dim=1)
    
    pred_group = preds[0]

    gt_group = group_gt[clip]

    pred_group_int = int(pred_group.cpu())  # <-- convert to int
    gt_group_int = int(gt_group)            # if gt_group is not already int


    # ---- frame-level drawing ----
    for frame_id, players in frames_ann.items():
        img_path = os.path.join(clip_path, f"{frame_id}.jpg")
        if not os.path.exists(img_path):
            continue

        img = read_image(img_path)

        boxes, labels = [], []
        for box_info in players:
            x1, y1, x2, y2 = box_info.box
            boxes.append([x1, y1, x2, y2])
            labels.append(box_info.category)  # ✅ GT player label only

        # draw players
        if len(boxes) > 0:
            img = draw_bounding_boxes(
                image=img,
                boxes=torch.tensor(boxes),
                labels=labels,
                colors="#FF00FF",
                width=3,
                font_size=30
            )

        # draw group GT vs prediction
        colors = ["#00FF00", "#00FF00" if gt_group == pred_group else "#FF0000"]
        
        img = draw_bounding_boxes(
            image=img,
            boxes=torch.tensor([
                [250, 0, 600, 50],
                [600, 0, 850, 50]
            ]),
            labels=[
                f"ground_truth: {ID_TO_CATEGORY[gt_group_int]}",
                f"predicted: {ID_TO_CATEGORY[pred_group_int]}"
            ],
            colors=colors,
            width=0,
            font_size=60
        )

        write_jpeg(img, os.path.join(save_clip, f"{frame_id}.jpg"))

[DEBUG] clip=105655, frames=9


/tmp/ipykernel_26574/2423073228.py:26: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  data = torch.tensor(data, dtype=torch.float32).to(device)
/tmp/ipykernel_26574/2423073228.py:27: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  target = torch.tensor(target, dtype=torch.long).to(device)
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/torchvision/utils.py:233: UserWarning: Argument 'font_size' will be ignored since 'font' is not set.
  warnings.warn("Argument 'font_size' will be ignored since 'font' is not set.")


[DEBUG] clip=105735, frames=9
[DEBUG] clip=105755, frames=9


[DEBUG] clip=105805, frames=9
[DEBUG] clip=105860, frames=9
[DEBUG] clip=105865, frames=9
[DEBUG] clip=105890, frames=9
[DEBUG] clip=105930, frames=9
[DEBUG] clip=105950, frames=9
[DEBUG] clip=105970, frames=9
[DEBUG] clip=106040, frames=9
[DEBUG] clip=106095, frames=9
[DEBUG] clip=106135, frames=9
[DEBUG] clip=106145, frames=9
[DEBUG] clip=115100, frames=9
[DEBUG] clip=115155, frames=9
[DEBUG] clip=115200, frames=9
[DEBUG] clip=115220, frames=9
[DEBUG] clip=115270, frames=9
[DEBUG] clip=115305, frames=9
[DEBUG] clip=115330, frames=9
[DEBUG] clip=115365, frames=9
[DEBUG] clip=130405, frames=9
[DEBUG] clip=130430, frames=9
[DEBUG] clip=130505, frames=9
[DEBUG] clip=130545, frames=9
[DEBUG] clip=130585, frames=9
[DEBUG] clip=130630, frames=9
[DEBUG] clip=130670, frames=9
[DEBUG] clip=133335, frames=9
[DEBUG] clip=133370, frames=9
[DEBUG] clip=133390, frames=9
[DEBUG] clip=133475, frames=9
[DEBUG] clip=133525, frames=9
[DEBUG] clip=133555, frames=9
[DEBUG] clip=133595, frames=9
[DEBUG] cl